#2024年度B3ゼミ AIでクイズを解いてみた!
## テキスト+コンテンツミニプロテーマ


### 0.Google Colabで無料のGPU(T4)を使う設定

画面上の「ランタイム」→ 「ランタイムタイプの変更」→「ハードウェアアクセラレーター」を「T4 GPU」に変更
### 変更して保存(一度ランタイムが切断されるので、コードは再実行する必要がある)

In [ ]:
!nvidia-smi
# GPUが認識されれば、表示される

Sun Nov 24 07:28:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### 1.必要なライブラリのインストール


In [ ]:
!pip install datasets transformers[torch,sentencepiece] trl peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.

### 2.設定、準備等


In [ ]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定
set_seed(42)

In [ ]:
from google.colab import drive

# Googleドライブを"drive"ディレクトリ以下にマウント
drive.mount("drive")

Mounted at drive


### データセットの準備

In [ ]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/aio-retrieverのリポジトリから
# AI王データセットを読み込む
dataset = load_dataset(
    "llm-book/aio-retriever", trust_remote_code=True
)

# 読み込まれたデータセットの形式と事例数を確認
print(dataset)

from pprint import pprint

# 読み込まれたデータセットの内容を確認
pprint(dataset["validation"][0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

aio-retriever.py:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22335 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
        num_rows: 22335
    })
    validation: Dataset({
        features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
        num_rows: 1000
    })
})
{'answers': ['ジェット団'],
 'competition': '第2回AI王',
 'negative_passage_indices': [1,
                              2,
                              4,
                              6,
                              8,
                              9,
                              10,
                              11,
                              13,
                              14,
                      

In [ ]:
from typing import Any

def filter_example(
    example: dict[str, Any], max_passages: int = 3
) -> bool:
    """上位max_passages件のパッセージに正例が含まれていない事例を除外"""
    if len(example["positive_passage_indices"]) == 0:
        return False
    if example["positive_passage_indices"][0] >= max_passages:
        return False

    return True

dataset = dataset.filter(filter_example)

Filter:   0%|          | 0/22335 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
def process_example(
    example: dict[str, Any], max_passages: int = 3
) -> dict[str, Any]:
    """質問、パッセージ、正解の組からプロンプトを作成し、会話データに変換"""

    # exampleから必要な情報を取得
    question = example["question"]
    answer = example["answers"][0]
    passages = [p["text"] for p in example["passages"]]

    # max_passages件のパッセージを選択
    passages = passages[:max_passages]

    messages: list[dict[str, str]] = []
    # プロンプトとパッセージをユーザのメッセージとして会話データに追加
    prompt_text = "".join(
        [
            "あなたには今からクイズに答えてもらいます。",
            "問題を与えますので、その解答のみを簡潔に出力してください。\n",
            "また解答の参考になりうるテキストを与えます。",
            "解答を含まない場合もあるのでその場合は無視してください。\n\n",
            "---\n",
            "\n\n".join(passages),
            "\n---\n\n",
            f"問題: {question}",
        ]
    )
    messages.append({"role": "user", "content": prompt_text})
    # LLMが出力すべき内容（クイズ問題の答え）を会話データに追加
    messages.append({"role": "assistant", "content": answer})

    # 会話データを事例の"messages"フィールドに追加
    example["messages"] = messages
    return example

dataset = dataset.map(
    process_example, remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/13951 [00:00<?, ? examples/s]

Map:   0%|          | 0/637 [00:00<?, ? examples/s]

In [ ]:
# 前処理後のデータセットの形式と事例数を確認
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 13951
    })
    validation: Dataset({
        features: ['messages'],
        num_rows: 637
    })
})


In [ ]:
# 前処理後のデータセットの内容を確認
pprint(dataset["validation"][0])

{'messages': [{'content': 'あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\n'
                          'また解答の参考になりうるテキストを与えます。解答を含まない場合もあるのでその場合は無視してください。\n'
                          '\n'
                          '---\n'
                          'ニューヨークのウエスト・サイド。午後5時。ポーランド系アメリカ人の少年非行グループ「ジェッツ」(ジェット団)と、新参のプエルトリコ系アメリカ人の少年非行グループ「シャークス」(シャーク団)は、なわばりを巡って対立している。今日も2グループの間で争いが起きるが警官の呼子笛の音に止められる(“Prologue”「プロローグ」)。クラプキ巡査とシュランク警部補が現れて少年たちに説教をして帰っていく。ジェッツのリーダー・リフはシャークスとの関係をはっきりさせるために決闘しようと言い出し、ジェッツのメンバーが賛成する。ついては決闘についての取り決めをシャークスとする必要があり、リフは自分の副官にトニーを選ぶ。メンバーは初めトニーはもう抜けたと反対するが、リフは(海兵隊のように)「一度ジェッツになったら死ぬまでジェッツだ」と歌う。\n'
                          '\n'
                          '『ウエストサイド物語』(ウエストサイドものがたり)は、宝塚歌劇団によるミュージカル作品。ブロードウェイ・ミュージカルの傑作『ウエストサイド物語』の日本での上演の一つである。\n'
                          '\n'
                          '『ウエスト・サイド物語』(ウエスト・サイドものがたり、West Side '
                          'Story)は、アーサー・ローレンツ脚本、レナード・バーンスタイン音楽、スティーヴン・ソンドハイム歌詞のブロードウェイ・ミュージカル。原案ジェローム・ロビンズ。1957年初演。『ウエスト・サイド・ストーリー』とも

### 4.モデルの定義・学習準備

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

# Hugging Face Hubにおけるモデル名を指定
base_model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja"

# モデルの量子化の設定
# 量子化をする場合としない場合で結果にどのような誤差があるのだろう? 時間があれば調べてね！
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビット量子化のパラメータを読み込む
    bnb_4bit_quant_type="nf4",  # NF4量子化を使用
    bnb_4bit_compute_dtype=torch.bfloat16,  # 計算時のデータ型としてBF16を使用
)



# モデルの量子化の設定を用いてモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,  # 量子化設定
    use_cache=False,  # 後にgradient checkpointingを有効にするために必要
    device_map="auto",
)

# トークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(base_model_name)


config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset
from tqdm.notebook import tqdm
from transformers import PreTrainedModel

def evaluate(
    model: PreTrainedModel, dataset: Dataset
) -> tuple[list[str], list[str], float]:
    """データセットの各問題に対するモデルの出力を評価し、正解率を算出"""
    pred_answers = []
    gold_answers = []
    num_correct = 0

    for example in tqdm(dataset):
        # プロンプトにチャットテンプレートを適用
        model_inputs = tokenizer.apply_chat_template(
            example["messages"][:-1],
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        # プロンプトの長さ（トークン数）を取得しておく
        input_length = model_inputs.shape[1]

        # モデルにプロンプトを入力し、出力を得る
        generated_ids = model.generate(
            model_inputs,
            max_new_tokens=32,
            do_sample=False,
            temperature=None,
            top_p=None,
        )

        # モデルの出力から答えの部分を文字列として取り出す
        pred_answer = tokenizer.batch_decode(
            generated_ids[:, input_length:], skip_special_tokens=True
        )[0]

        # 正解の文字列を取り出す
        gold_answer = example["messages"][-1]["content"]

        # モデルの答えと正解が一致していれば正答とカウント
        if pred_answer == gold_answer:
            num_correct += 1

        # モデルの答えと正解をそれぞれリストに追加
        pred_answers.append(pred_answer)
        gold_answers.append(gold_answer)
        # 正解率を計算
    accuracy = num_correct / len(pred_answers)

    return pred_answers, gold_answers, accuracy

In [ ]:
# 指示チューニング前のモデルを使って評価
"""
pred_answers, gold_answers, accuracy = evaluate(
    model, dataset["validation"]
)
"""

# 無料版ColabのT4 GPUなどでは評価に時間を要するため、最初の100事例のみで評価
pred_answers, gold_answers, accuracy = evaluate(
    model, dataset["validation"].take(100)
)

print(f"正解率: {accuracy:.1%}")

  0%|          | 0/100 [00:00<?, ?it/s]

正解率: 54.0%


In [ ]:
# モデルが予測した答えを表示
for pred_answer, gold_answer in zip(
    pred_answers[:20], gold_answers[:20]
):
    print(f"正解: {gold_answer} / 予測: {pred_answer}")

# 今のLLMはどんな問題が得意でどんな問題が不得意か、回答から分析してみよう

正解: ジェット団 / 予測: ジェッツ団
正解: コマイ / 予測: スケトウダラ
正解: START / 予測: START
正解: ニュートン / 予測: アイザック・ニュートン
正解: 天平文化 / 予測: 聖武天皇の時代に栄えた、東大寺正倉院や唐招提寺金堂など、中国・唐の影響を強く受け
正解: アメリカンリーグ / 予測: アメリカンリーグ
正解: 華道 / 予測: 池坊、草月流、小原流は、日本の伝統的な生け花の三大流派である。池坊は伝統
正解: ラストベルト / 予測: ラストベルト
正解: 天童市 / 予測: 天童市
正解: 医学部 / 予測: 安部公房は東京大学医学部出身。
正解: 村田珠光 / 予測: 山上宗二は、「侘び茶」の創始者として知られる室町時代の茶人である。彼は
正解: 23時 / 予測: 日本のテレビ業界で「プライムタイム」といえば、毎日19時から23時までの時間帯のことです。
正解: 佐々木彩夏 / 予測: 佐々木彩夏
正解: 早口言葉 / 予測: 英語で「タングツイスター」という言葉遊びは「早口言葉」です。
正解: 昭和基地 / 予測: 昭和基地
正解: 開口一番 / 予測: 「開口一番」
正解: マクベス / 予測: マクベス
正解: ニ長調 / 予測: ト短調
正解: 版籍奉還 / 予測: 版籍奉還
正解: IBS / 予測: IBS


In [ ]:
# 訓練セットのすべての事例にチャットテンプレートを適用
tokenized_train_dataset = [
    tokenizer.apply_chat_template(example["messages"])
    for example in dataset["train"]
]


from trl import DataCollatorForCompletionOnlyLM

# collate関数を初期化
bos = tokenizer.bos_token
collator = DataCollatorForCompletionOnlyLM(
    # ユーザとアシスタントそれぞれの発話開始文字列
    instruction_template=bos + "ユーザ：",
    response_template=bos + "アシスタント：",
    tokenizer=tokenizer,  # トークナイザ
)

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

# LoRAの設定
peft_config = LoraConfig(
    r=128,  # 差分行列のランク
    lora_alpha=128,  # LoRA層の出力のスケールを調整するハイパーパラメータ
    lora_dropout=0.05,  # LoRA層に適用するドロップアウト
    task_type=TaskType.CAUSAL_LM,  # LLMが解くタスクのタイプを指定
    # LoRAで学習するモジュール
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

model.enable_input_require_grads()  # 学習を行うために必要
model = get_peft_model(model, peft_config)  # モデルにLoRAを適用
model.print_trainable_parameters()  # 学習可能なパラメータ数を表示

trainable params: 319,815,680 || all params: 7,149,785,088 || trainable%: 4.4731


### 5.学習の実行/モデルの評価

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

# 訓練のハイパーパラメータを設定
"""
training_args = TrainingArguments(
    output_dir="./drive/MyDrive/llm_book/RAG_IT_results",  # 結果の保存フォルダ
    bf16=True,  # BF16を使用した学習の有効化
    max_steps=100,  # 訓練ステップ数
    per_device_train_batch_size=2,  # 訓練時のバッチサイズ
    gradient_accumulation_steps=8,  # 勾配累積のステップ数（5.5.2節）
    gradient_checkpointing=True,  # 勾配チェックポインティングの有効化（5.5.3節）
    optim="paged_adamw_8bit",  # 最適化器
    learning_rate=1e-4,  # 学習率
    lr_scheduler_type="cosine",  # 学習率スケジューラの種類
    max_grad_norm=0.3,  # 勾配クリッピングにおけるノルムの最大値（9.4.3節）
    warmup_ratio=0.1,  # 学習率のウォームアップの長さ（5.2.8節）
    logging_steps=10,  # ロギングの頻度
    save_steps=50,  # モデルの保存頻度
    report_to="none",  # 外部ツールへのログを無効化
)
"""

# 無料版のT4 GPUなど、低メモリ環境での学習パラメータ
# バッチサイズと勾配累積のステップ数を小さく設定
training_args = TrainingArguments(
    output_dir="./drive/MyDrive/llm_book/RAG_IT_results",  # 結果の保存フォルダ
    bf16=True,  # BF16を使用した学習の有効化
    max_steps=100,  # 訓練ステップ数
    per_device_train_batch_size=1,  # 訓練時のバッチサイズ
    gradient_accumulation_steps=4,  # 勾配累積のステップ数（5.5.2節）
    gradient_checkpointing=True,  # 勾配チェックポインティングの有効化（5.5.3節）
    optim="paged_adamw_8bit",  # 最適化器
    learning_rate=1e-4,  # 学習率
    lr_scheduler_type="cosine",  # 学習率スケジューラの種類
    max_grad_norm=0.3,  # 勾配クリッピングにおけるノルムの最大値（9.4.3節）
    warmup_ratio=0.1,  # 学習率のウォームアップの長さ（5.2.8節）
    logging_steps=10,  # ロギングの頻度
    save_steps=50,  # モデルの保存頻度
    report_to="none",  # 外部ツールへのログを無効化
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir = "./drive/MyDrive/llm_book/logs"
    )

# Trainerを初期化
trainer = Trainer(
    model,
    train_dataset=tokenized_train_dataset,  # トークンID化されたデータセット
    data_collator=collator,  # ラベルの加工及びミニバッチ構築処理を行うモジュール
    args=training_args,  # 訓練の設定
    tokenizer=tokenizer,  # パラメータ保存時にトークナイザも一緒に保存するために指定
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# モデルの訓練を実行
trainer.train()


<ipython-input-16-c114b79e1739>:43: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
10,0.487200
20,0.215300
30,0.130100
40,0.077700
50,0.191400
60,0.097600
70,0.264600
80,0.047900
90,0.221700
100,0.081400


TrainOutput(global_step=100, training_loss=0.18148509562015533, metrics={'train_runtime': 6408.3158, 'train_samples_per_second': 0.062, 'train_steps_per_second': 0.016, 'total_flos': 1.1585756616966144e+16, 'train_loss': 0.18148509562015533, 'epoch': 0.028671779800731132})

In [ ]:
# 指示チューニング後のモデルを使って評価
pred_answers, gold_answers, accuracy = evaluate(
    model, dataset["validation"].take(100)
)

print(f"正解率: {accuracy:.1%}")

  0%|          | 0/637 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
# モデルが予測した解答を表示
for pred_answer, gold_answer in zip(
    pred_answers[:20], gold_answers[:20]
):
    print(f"正解: {gold_answer} / 予測: {pred_answer}")